## Parkolóház

Adott a következő adatbázis:

  ![diag](.\diag.png)

Írjunk tárolt eljárást, amely a parkolásokat menedzseli. A feladat megoldásához használjunk tranzakció kezelést, valamint megfelelően kezeljük a hibákat.

Paraméterek:

- Bemeneti paraméterként megkapja egy parkolóház ID-ját, egy kezdeti es egy végső időpontot, valamint egy rendszámot
    
- Kimeneti paraméterként visszaadja a parkolás összköltséget
    

### Feladatai

- Ellenőrzi, hogy a megadott parkolóház ID létezik-e es van szabad férőhely(Parkolóház.Férőhely\>0) ha nem, akkor hiba üzenetet dob, valamint -1-gyel tér vissza (pl. 'Nincs ilyen parkolóház!')
    
- Ellenőrzi, hogy a megadott kezdeti es végső időpontok helyesek-e (megfelelő sorrendben vannak), ha nem, akkor hiba üzenetet dob (pl. 'Hibás időpontok!') -2-vel tér vissza
    
- Ellenőrzi, hogy a megadott rendszam létezik-e, ha nem akkor felvezeti a rendszámot a Járművek táblába az 'Anonim' nevű klienshez
    

Ezután a parkolás felvezetése (az EgyediParkolasok táblába) 3 módon történhet: bérlettel rendelkező, bérlet nélküli vagy Anonim ügyfél

- Bérlettel rendelkezik az ügyfél, ha a HonaposParkoloBerletek táblában van olyan sor, amelynek a Kliense az illető jármű tulajdonosa, a BerletKezdete és a BerletVege között van a parkolás kezdeti és végső időpontja és a parkolóházba szól. Ebben az esetben a parkolás költsége 0 (az alap parkolási díj és a fizetett díj is)
    
- Ha a kliens anonim az alapdíj 2000 valamint ha nem rendelkezik bérlettel az alapdíj 1000 lesz. A parkolás költségét (Fizetettdíj) a parkolás kezdeti és végső időpontja közötti időtartam alapján számoljuk ki az alábbiak szerint:
    
    - 500/óra ha a parkolás kezdeti és végső időpontja közötti időtartam 24 óra vagy annál kevesebb + az alapdíj
    - 8000/nap különben + az alapdíj

  A parkolás beszúrása után a parkolóházban a szabad helyek szamat csökkenti 1-gyel (Parkolohaz.Ferohely)  

**Hívjuk is meg az eljárást és jelenítsük meg a kimeneti paramétert és visszatérési értéket!**

**Minimális követelmény**

- A tárolt eljárás létrehozása es meghivasa
- 1 paraméter ellenőrzése hiba kezeléssel
- Tranzakció hirdetése és minden ágon való befejezése
- Parkolás sor beszúrása

In [68]:
CREATE OR ALTER PROCEDURE spParkolos
@ParkoloHazID INT,
@KezdetiIdo DATETIME,
@VegzesiIdo DATETIME,
@Rendszam VARCHAR(30),
@Osszkoltseg DECIMAL OUTPUT
AS
BEGIN
    SET NOCOUNT ON;
    SET TRANSACTION ISOLATION LEVEL SERIALIZABLE;
    
    DECLARE @Ferohely INT
    DECLARE @Kliens INT, @berlet int, @alapdij DECIMAL, @idopontokKulonbseg DECIMAL, @TULID INT

    BEGIN TRY
        BEGIN TRANSACTION
        set @Osszkoltseg=0
        SELECT @Ferohely = Ferohely FROM ParkoloHazak WHERE ParkoloHazID = @ParkoloHazID;

--LETEZIK E A PARKOLOHAZ, HA IGEN, VAN E BENNE URES HELY
        IF NOT EXISTS (SELECT 1 FROM ParkoloHazak WHERE ParkoloHazID = @ParkoloHazID)
        BEGIN
            RAISERROR('NINCS ILYEN PARKOLOHAZ', 16, 1)
            RETURN -1
        END
        ELSE IF @Ferohely <= 0
        BEGIN
            RAISERROR('NINCS SZABAD FEROHELY A PARKOLOHAZBAN', 16, 1)
            RETURN -1
        END

--IDOPONTOK ELLENORZESE
        IF @KezdetiIdo>=@VegzesiIdo
        BEGIN
            RAISERROR('HIBAS IDOPONTOK',16,1)
            RETURN -2
        END

--LETEZIK E A RENDSZAM
        IF NOT EXISTS (SELECT 1 FROM Jarmuvek WHERE RendSzam=@Rendszam)
        BEGIN
            SELECT @Kliens=KliensID FROM Kliensek WHERE Nev='Anonim'
            INSERT INTO Jarmuvek(KliensID,RendSzam) VALUES (@Kliens,@Rendszam)
        END

        SELECT @berlet=BerletID FROM HonaposParkoloBerletek
        WHERE ParkoloHazID=@ParkoloHazID 
        AND KezdoIdopont<=@KezdetiIdo
        AND VegsoIdopont>=@VegzesiIdo
        AND KliensID=(SELECT KliensID FROM Jarmuvek WHERE RendSzam=@Rendszam)

        IF @berlet IS NOT NULL
        BEGIN
            SET @alapdij=0
        END
        ELSE IF @TULID=(SELECT KliensID FROM Kliensek WHERE Nev='Anonim')
        BEGIN
            SET @alapdij=2000
            SET @Osszkoltseg =CASE
							WHEN DATEDIFF(HOUR,@KezdetiIdo,@VegzesiIdo)<24 THEN 500*DATEDIFF(HOUR,@KezdetiIdo,@VegzesiIdo)+@alapdij
							ELSE 8000+@alapdij
							END
        END
        ELSE
        BEGIN
            SET @alapdij=1000
            SET @Osszkoltseg =CASE
							WHEN DATEDIFF(HOUR,@KezdetiIdo,@VegzesiIdo)<24 THEN 500*DATEDIFF(HOUR,@KezdetiIdo,@VegzesiIdo)+@alapdij
							ELSE 8000+@alapdij
							END
        END
            INSERT INTO EgyszeriParkolasok VALUES ((SELECT JarmuID FROM Jarmuvek WHERE RendSzam=@RendSzam),
			@ParkoloHazID,@KezdetiIdo,@VegzesiIdo,@alapdij,@Osszkoltseg)
			UPDATE ParkoloHazak 
			SET Ferohely=Ferohely-1
			WHERE ParkoloHazID=@ParkoloHazID

        COMMIT TRANSACTION
    END TRY

    BEGIN CATCH
        SELECT
            ERROR_MESSAGE() AS Errormessage,
            ERROR_NUMBER() AS Errornumber,
            ERROR_SEVERITY() AS Errorseverity,
            ERROR_STATE() AS Errorstate;
        ROLLBACK
    END CATCH
END

Commands completed successfully.

Total execution time: 00:00:00.009

In [65]:
select * from HonaposParkoloBerletek
select * from Jarmuvek
select * from ParkoloHazak
select * from Kliensek
select * from EgyszeriParkolasok

(12 rows affected)

(10 rows affected)

(5 rows affected)

(7 rows affected)

(7 rows affected)

...

BerletID,KliensID,ParkoloHazID,VasarlasDatuma,KezdoIdopont,VegsoIdopont,Dij
1,2,1,2021-06-29,2021-06-29 15:00:00.000,2021-07-29 15:00:00.000,300.00
2,2,4,2021-08-30,2021-08-30 16:30:00.000,2021-09-30 16:30:00.000,250.00
3,4,2,2021-11-13,2021-11-17 08:00:00.000,2021-12-16 18:00:00.000,450.00
4,4,2,2023-12-15,2023-12-17 08:00:00.000,2024-01-16 18:00:00.000,450.00
5,4,2,2022-01-17,2022-01-17 08:00:00.000,2022-02-16 18:00:00.000,450.00
6,5,2,2021-09-01,2021-09-02 08:00:00.000,2021-10-01 18:00:00.000,450.00
7,5,3,2023-10-02,2023-10-02 09:00:00.000,2023-11-01 18:00:00.000,350.00
8,3,3,2022-01-12,2022-01-15 08:00:00.000,2022-02-14 08:00:00.000,350.00
9,6,2,2023-11-12,2023-11-12 08:00:00.000,2023-12-11 18:00:00.000,450.00
10,1,2,2023-12-14,2023-12-14 08:00:00.000,2024-02-16 18:00:00.000,450.00


JarmuID,KliensID,RendSzam
1,2,CJ 01 ABX
2,1,CJ 32 ABC
3,4,SM 12 FGT
4,4,MS 17 PAX
5,3,B 04 EFT
6,2,HR 05 YSR
7,5,CV 12 BAW
8,5,CJ 09 SKY
9,6,BH 14 UGE
10,5,CJ 06 SKY


ParkoloHazID,Ferohely,Cim,CegNev,MukodikEjszaka,MinFizetendoOraSzam,HonaposBerletElerheto,HonaposBerletiDij
1,164,"Foter 14, Kolozsvar",Kolozsvari Parkolohaz,0,1,1,300.00
2,100,"Beke ter 37, Kolozsvar",Cool Parking,1,0.5,1,450.00
3,40,Hosszu ut 56,Best Parking,1,1,1,350.00
4,1030,Monostor 198,Affordable PArking,1,1,1,250.00
5,0,Kalacs ut 23,Maskara KFT,1,1,1,400.00


KliensID,RegisztracioDatuma,TelSzam,Nev,Cim
1,1970-01-01,----,Anonim,----
2,2020-12-13,0740-123456,Toth Adam,Hosszu ut 24
3,2021-06-24,+40-745-456456,Horvath Denes,Kossuth utca 19/24
4,2022-01-16,0746123456,Szabo Annamari,Rozsak tere 6
5,2021-08-08,0745654321,Lakatos Lajos,Petofi utca 78B
6,2021-01-01,0743-909089,Bertalan Lilla,Farkas utca 1
7,2021-11-11,0722-786756,Tamasi Virag,Kis utca 3


ParkolasID,JarmuID,ParkoloHazID,KezdoIdopont,VegsoIdopont,AlapParkolasiDij,FizetettDij
1,10,1,2022-01-24 15:04:00.000,2022-01-24 16:30:40.000,7.00,12.00
2,10,1,2021-02-25 13:56:00.000,2021-02-25 14:35:40.000,7.00,12.00
3,6,3,2021-11-14 19:16:00.000,2021-11-14 20:30:40.000,9.00,20.00
4,7,3,2021-09-04 09:08:00.000,2021-09-04 11:58:00.000,10.00,50.00
5,5,2,2022-01-22 12:04:00.000,2022-01-22 15:25:40.000,8.00,30.00
6,2,2,2022-01-23 14:07:00.000,2022-01-23 15:47:30.000,8.00,25.00
7,2,1,2021-06-11 18:00:00.000,2021-06-11 19:00:00.000,2000.00,2500.00


In [4]:
DECLARE @Osszkoltseg DECIMAL
EXEC spParkolos 
@ParkoloHazID = 2,
@KezdetiIdo='2023-12-19 09:00:00.000',
@VegzesiIdo='2023-12-28 20:30:40.000',
@Rendszam= 'SM 12 FGT',
@Osszkoltseg=@Osszkoltseg OUT
SELECT @Osszkoltseg AS Osszkoltseg


(1 row affected)

Total execution time: 00:00:00.015

Osszkoltseg
0


In [5]:
select * from EgyszeriParkolasok
select * from ParkoloHazak

(11 rows affected)

(5 rows affected)

Total execution time: 00:00:00.017

ParkolasID,JarmuID,ParkoloHazID,KezdoIdopont,VegsoIdopont,AlapParkolasiDij,FizetettDij
1,10,1,2022-01-24 15:04:00.000,2022-01-24 16:30:40.000,7.00,12.00
2,10,1,2021-02-25 13:56:00.000,2021-02-25 14:35:40.000,7.00,12.00
3,6,3,2021-11-14 19:16:00.000,2021-11-14 20:30:40.000,9.00,20.00
4,7,3,2021-09-04 09:08:00.000,2021-09-04 11:58:00.000,10.00,50.00
5,5,2,2022-01-22 12:04:00.000,2022-01-22 15:25:40.000,8.00,30.00
6,2,2,2022-01-23 14:07:00.000,2022-01-23 15:47:30.000,8.00,25.00
7,2,1,2021-06-11 18:00:00.000,2021-06-11 19:00:00.000,2000.00,2500.00
8,2,1,2021-06-11 18:00:00.000,2021-06-11 19:00:00.000,2000.00,2500.00
9,4,2,2024-01-24 15:04:00.000,2024-01-24 20:30:40.000,1000.00,3500.00
10,3,2,2023-12-17 09:00:00.000,2023-12-17 20:30:40.000,0.00,0.00


ParkoloHazID,Ferohely,Cim,CegNev,MukodikEjszaka,MinFizetendoOraSzam,HonaposBerletElerheto,HonaposBerletiDij
1,163,"Foter 14, Kolozsvar",Kolozsvari Parkolohaz,0,1,1,300.00
2,97,"Beke ter 37, Kolozsvar",Cool Parking,1,0.5,1,450.00
3,40,Hosszu ut 56,Best Parking,1,1,1,350.00
4,1030,Monostor 198,Affordable PArking,1,1,1,250.00
5,0,Kalacs ut 23,Maskara KFT,1,1,1,400.00


In [47]:
INSERT INTO ParkoloHazak(Ferohely, Cim, CegNev, MukodikEjszaka,MinFizetendoOraSzam,HonaposBerletElerheto,HonaposBerletiDij)
VALUES (0, 'Kalacs ut 23', 'Maskara KFT', 1, 1, 1, 400)

(1 row affected)

Total execution time: 00:00:00.007

In [48]:
SELECT * FROM ParkoloHazak

(5 rows affected)

Total execution time: 00:00:00.008

ParkoloHazID,Ferohely,Cim,CegNev,MukodikEjszaka,MinFizetendoOraSzam,HonaposBerletElerheto,HonaposBerletiDij
1,165,"Foter 14, Kolozsvar",Kolozsvari Parkolohaz,0,1,1,300.00
2,100,"Beke ter 37, Kolozsvar",Cool Parking,1,0.5,1,450.00
3,40,Hosszu ut 56,Best Parking,1,1,1,350.00
4,1030,Monostor 198,Affordable PArking,1,1,1,250.00
5,0,Kalacs ut 23,Maskara KFT,1,1,1,400.00
